In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from UnarySim.sw.bitstream.gen import RNG, BSGen


In [3]:
rng = RNG(4, 2, "Sobol").cuda()
print(rng)

rng1 = RNG(4, 2, "Race")
print(rng1)

RNG()
RNG()


In [4]:
rng()

Parameter containing:
tensor([ 8,  4, 12,  6, 14,  2, 10,  5, 13,  1,  9,  3, 11,  7, 15,  7],
       device='cuda:0')

In [5]:
bs1 = BSGen(torch.tensor([14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14]).cuda(), rng()).cuda()

In [6]:
bs1(rng())

tensor([1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0',
       dtype=torch.int8)

In [7]:
bs2 = BSGen(torch.tensor([14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14]), rng1())

In [8]:
bs2(rng1())

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0], dtype=torch.int8)